In [1]:
import numpy as np
import pandas as pd
from mylibs import tools, functions as f

In [2]:
%matplotlib qt

In [4]:
exp = tools.Experiment()
exp.load_csv(tools.input_path())
exp.set_info(
    compound=input('Compound: '),
    rho=float(input('Rho: ')),
    w=float(input('W mass: ')),
)
tools.temporal_plot(exp,ylabel='Viscosity [cP]')


In [19]:
tmp=tools.Experiment()
tmp.load_hdf5(tools.input_path())

,time,y,Viscosity_verbose,x,Temperature_verbose
0,1.0,831.980284,OK,13.8,OK
1,2.0,831.980284,OK,13.8,OK
2,3.0,831.980284,OK,13.8,OK
3,4.0,830.775912,image_sweep_check,13.8,OK
4,5.0,830.775912,image_sweep_check,13.8,OK
...,...,...,...,...,...
18502,19333.0,6863.238512,OK,40.5,OK
18503,19334.0,6818.478261,OK,40.5,OK
18504,19335.0,6878.289474,OK,40.5,image_sweep_check
18505,19336.0,6863.238512,OK,40.5,OK


In [20]:
tmp.info

{'compound': 'ocm',
 'rho': 1.72,
 'w': 0.0,
 'D0': 58053559827657.89,
 'f_statistic': 3029495.381638645,
 'r2': 0.9951642727866552,
 'E': 59372.593499571696,
 'dD0': 2256959362103.371,
 'dE': 95.76658562539887}

In [21]:
tmp.log

[('csv loaded',
  'D:\\WORKS\\Diplom-work\\Experiments\\OCM_viscosity\\OCM0\\data_OCM0_up.csv'),
 ('initial_filter', {'time': [0.0, inf], 'y': [40.0, 350.0], 'x': (12, 42)}),
 ('C_to_K', []),
 ('nu_D', []),
 ('linearize', []),
 ('iqr_filter', []),
 ('delinearize', []),
 ('K_to_C', []),
 ('hdf5 loaded',
  'D:\\Works\\Diplom-work\\Experiments\\OCM_viscosity\\OCM0\\data_OCM0_up.hdf5')]

In [5]:
exp = tools.configurate_data(exp)

In [18]:
exp.info

{}

In [16]:
exp.log

[('csv loaded',
  'D:\\WORKS\\Diplom-work\\Experiments\\OCM_viscosity\\OCM0\\data_OCM0_up.csv'),
 ('initial_filter', {'time': [0.0, inf], 'y': [40.0, 350.0], 'x': (12, 42)}),
 ('C_to_K', []),
 ('nu_D', []),
 ('linearize', []),
 ('iqr_filter', []),
 ('delinearize', []),
 ('K_to_C', []),
 ('hdf5 loaded',
  'D:\\Works\\Diplom-work\\Experiments\\OCM_viscosity\\OCM0\\data_OCM0_up.hdf5')]

In [33]:
tmp_exp =exp.copy()

In [34]:
tools.temperature_plot(
    exp,
    title='Viscosity',
    xlabel='Temperature [C]',
    ylabel='Viscosity [cP]',
    interactive=True,
    save=True,
)


In [35]:
exp.apply(f.C_to_K)
exp.apply(f.nu_D)
exp.apply(f.linearize)
exp.group_filter(f.iqr_filter)
exp.apply(f.delinearize)
exp.apply(f.nu_D)
exp.apply(f.K_to_C)
print('Filtered')
tools.temperature_plot(
    exp,
    title='Viscosity clear',
    xlabel='Temperature [C]',
    ylabel='Viscosity [cP]',
    interactive=True,
    save=True,
)

Filtered


In [39]:
exp.apply(f.C_to_K)
exp.apply(f.nu_D)
exp.apply(f.K_to_C)
tools.temperature_plot(
    exp,
    title='Diffusion',
    xlabel='Temperature [C]',
    ylabel='D [m2/s]',
    interactive=True,
    save=True
)

In [27]:
info, result, func = tools.regress(exp)
exp.set_info(**info)
x = np.linspace(13, 42, 100) + 273.15
ols_res = tools.Experiment(
    pd.DataFrame({
        'x': x, 'y': func(x), 'time': x * 0
    }),
    'interpolated',
)

ols_res.apply(f.K_to_C);

In [28]:
exp2= exp.copy()
ols_res2 = ols_res.copy()
exp2.apply(f.C_to_K)
ols_res2.apply(f.C_to_K)
exp2.apply(f.linearize)
ols_res2.apply(f.linearize)

tools.comparation_plot(
    exp2,
    ols_res2,
    title='OLS_Linear',
    xlabel='Temperature',
    ylabel='D',
    interactive=True,
    save=True,
)

In [30]:
tools.comparation_plot(
    exp,
    ols_res,
    title='OLS_Diffusion',
    xlabel='Temperature [C]',
    ylabel='D [m2/s]',
    interactive=True,
    save=True,
)

In [50]:
exp.save_hdf5()

In [40]:
exp.d

,time,y,Viscosity_verbose,x,Temperature_verbose
0,1.0,831.980284,OK,13.8,OK
1,2.0,831.980284,OK,13.8,OK
2,3.0,831.980284,OK,13.8,OK
3,4.0,830.775912,image_sweep_check,13.8,OK
4,5.0,830.775912,image_sweep_check,13.8,OK
...,...,...,...,...,...
18502,19333.0,6863.238512,OK,40.5,OK
18503,19334.0,6818.478261,OK,40.5,OK
18504,19335.0,6878.289474,OK,40.5,image_sweep_check
18505,19336.0,6863.238512,OK,40.5,OK
